In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from vtlearn.genetic import SymbolicRegressor, SymbolicTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from vtlearn.functions import make_function, _extended_function_map
from vtlearn.fitness import make_fitness
from scipy.stats import spearmanr

from vtlearn.utils import GenerateDataByDataFrame

In [3]:
StockPrice = pd.read_pickle('fundemental_data_valid.pickle')
StockPrice.tail(3)

,Date,Pid,Open,High,Low,Close,PctChg,Wide,DealVolume,DealCap,...,EBITDA_Ratio,ROA,ROE,CF_Ratio,EPS,DPZ–Score,Z–Score,CMoney-Score,TR,Next10dayReturn
1943164,20230904,9958,160.0,162.5,156.0,159.0,0.00,4.09,4076.0,648003.0,...,18.25,0.64,2.09,2.1,1.3,1.44,1.39,8.0,4.09,0.0
1943165,20230905,9958,159.0,166.0,158.5,165.0,3.77,4.72,6347.0,1037655.0,...,18.25,0.64,2.09,2.1,1.3,1.44,1.39,8.0,7.00,0.0
1943166,20230906,9958,168.0,173.5,165.5,166.0,0.61,4.85,8467.0,1435955.0,...,18.25,0.64,2.09,2.1,1.3,1.44,1.39,8.0,8.50,0.0


In [4]:
train_data = StockPrice.fillna(0) ###
train_data = train_data.replace([np.inf, -np.inf, np.nan], 0.)

In [4]:
X, Y, W, column_names = GenerateDataByDataFrame(train_data, tgt='Next10dayReturn',
sequence_col=['Date'],group_col=['Pid']
) 

In [5]:
# 訓練模型

In [6]:
total_result = pd.read_pickle('result.pickle')

In [8]:
for gen in [30,40,50]:
    for x in [1,2,3,4,5,6,7,8,9,10]:
        for m in ['scipy-rank_ic']:
            st = SymbolicTransformer(generations=gen, 
                            population_size=1000, 
                            n_components=10, 
                            hall_of_fame=100, 
                            function_set=_extended_function_map, 
                            parsimony_coefficient='auto', 
                            tournament_size=20,
                            init_depth=(2, 6), 
                            metric=m, 
                            const_range=(-1.0, 1.0), 
                            p_crossover=0.4, 
                            p_subtree_mutation=0.1, 
                            p_hoist_mutation=0.1,
                            p_point_mutation=0.1, 
                            p_point_replace=0.3,                     
                            max_samples=0.8, 
                            oob_method='tail',
                            stopping_criteria = 1.0,
                            verbose=False,
                            random_state=x, 
                            n_jobs=5,
                            feature_names = column_names,
                            )

            st.fit_transform(X, Y)

            res = pd.DataFrame({
                'Formula':st[-1].__str__(),
                'Fitness_In':st[-1].raw_fitness_,
                'Fitness_Out':st[-1].oob_fitness_,
                'Metrics':m,
                'Generation':gen,
                'seed':x
                },index=[0])

            print(f'{st[-1].__str__()}  {st[-1].raw_fitness_}\n')

            total_result = pd.concat([total_result, res])
            total_result.to_pickle('result.pickle')

min(ROE, inv(Wide))  0.0655986137644937

sub(div(ROE, PB_Ratio), Wide)  0.07170125791426975

min(inv(Wide), ROE)  0.0655986137644937

div(div(ROE, PB_Ratio), PB_Ratio)  0.06503500100538341

div(div(cross_rank(ROE), PB_Ratio), Wide)  0.07539436978241373

div(div(cross_rank(ROE), PB_Ratio), Wide)  0.07539436978241373

sub(sub(ROE, Wide), PB_Ratio)  0.0690893705112928

inv(ts_max_5(Wide))  0.060768669827466315

div(div(ROE, Wide), PB_Ratio)  0.0705974013440255

div(cross_rank(ROE), ts_max_5(TR))  0.06900441285792279

min(ROE, inv(Wide))  0.0655986137644937

sub(div(ROE, PB_Ratio), Wide)  0.07170125791426975

min(inv(Wide), ROE)  0.0655986137644937

